# Floating Coupon Bond

The goal of the project is to compute the market value of a portfolio of floating coupon bonds taking into account the issuer credit risk. The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* A set of forward libor rates and their fixing dates
* A set of survival probabilities and the recovery rate of the issuer
* The static data of the pool of bonds to price (nominal, start date, end date, current coupon, margin, payment frequency)

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

### Inputs

In [5]:
import pandas as pd
from datetime import date 

today = date(2019, 10, 31)

libor_tenor = 6 

ois_quotes = pd.read_csv("ois_quotes.csv").to_dict('records')

df = pd.read_csv("survival_probabilities_proj5.csv")
df['date'] = pd.to_datetime(df['date']).dt.date
survival_probabilities = df.to_dict('records')

df = pd.read_csv("bonds_to_price_proj5.csv")
df['start_date'] = pd.to_datetime(df['start_date']).dt.date
df['end_date'] = pd.to_datetime(df['end_date']).dt.date
bonds_to_price = df.to_dict('records')

## Solution
The first step is to determine the discoutn curve using the bootstrap method.
So define a set of OIS according to the market quotes given in inputs.

In [6]:
from finmarkets import OvernightIndexSwap, generate_swap_dates
#from project_input import today, ois_quotes

ois_pillar_dates = [today]

swaps = []

for quote in ois_quotes:
    
    Swap = OvernightIndexSwap( 
                1e6, 
                generate_swap_dates(today,quote['maturity']),
                quote['rate']
            )
    
    swaps.append(Swap)
    ois_pillar_dates.append(Swap.payment_dates[-1])
    
ois_pillar_dates = sorted(ois_pillar_dates)
n_df_vector = len(ois_pillar_dates)

Then define the objective function with a discount curve with unknown discount factors $\vec{x}$ and the squared sum of the swap NPVs to minimize.

In [7]:
from finmarkets import DiscountCurve

def objective_function(x): 
    
    curve = DiscountCurve(today, ois_pillar_dates, x)
    
    sum_sq = 0.0 
    
    for swap in swaps: 
        sum_sq += swap.npv(curve)**2
        
    return sum_sq

Set initial values and boundaries of the unknown factors (remember that the first factor has to be 1 since refers to today's discount).

In [8]:
from scipy.optimize import minimize 

x0 = [1.0 for i in range(len(ois_pillar_dates))]

bounds = [(0.01,100.0) for i in range(len(ois_pillar_dates))]
bounds[0]= (1.0,1.0)

result = minimize(objective_function, x0, bounds = bounds)

Check the results to be sure the minimization worked...

In [9]:
print (result)

      fun: 0.0006561289556596521
 hess_inv: <31x31 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.79535026e+05, -2.83218934e-04, -3.24325939e-04, -3.24159579e-04,
       -3.11566571e-04, -3.47202302e-04, -3.37286557e-04, -2.87379278e-04,
       -3.00611012e-04, -1.85489428e-04, -1.04252349e-04, -1.83492739e-04,
       -2.79534460e-03,  2.00303036e-03,  2.00938521e-03,  1.81469677e-03,
        5.20176784e-04,  2.38865056e-03,  2.94091578e-03,  7.65019818e-04,
       -1.58840020e-03, -3.03431233e-03, -1.90878246e-03, -4.69327008e-04,
        8.17669782e-04,  1.48083477e-03,  1.45731172e-03, -1.18335236e-03,
        1.67383136e-03,  2.55535285e-04, -1.12756848e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 448
      nit: 11
   status: 0
  success: True
        x: array([1.        , 0.99991167, 0.99980687, 0.99970619, 0.99961015,
       0.99950111, 0.999392  , 0.99928207, 0.99916713, 0.99903027,
       0.998895  , 0.99874906, 0.99860278, 0.9980

In [10]:
print ("Initial value of obj_func {}".format(objective_function(x0)))
print ("Final value of obj_func {}".format(objective_function(result.x)))

Initial value of obj_func 1398807951100.5293
Final value of obj_func 0.0006561289556596521


Finally create the discount curve to be used in the exercise.

In [11]:
from finmarkets import DiscountCurve
from datetime import date 

eonia_curve = DiscountCurve(today, 
                            ois_pillar_dates, 
                            result.x)

eonia_curve.df(date(2030,10,31))

0.6710110388246098

Our bonds are indexed to a LIBOR 6m curve given in input so we need to create a `ForwardRateCurve` object to use it.

In [14]:
from finmarkets import ForwardRateCurve
from datetime import date

df = pd.read_csv("euribor6M_proj5.csv")
forward_rate_curve = ForwardRateCurve(pd.to_datetime(df['dates']).dt.date.to_list(), 
                                     df['rates'])
                
forward_rate_curve.forward_rate(date(2025,7,30))

0.014197814207650273

In a similar way we need to create a `CreditCurve` from the inputs.

In [15]:
from finmarkets import CreditCurve
from datetime import date

pillar_ndps = [1.0]
ndps_pillar_dates = [today]

for ndps in survival_probabilities:
    pillar_ndps.append(ndps['ndp'])
    
for dates in survival_probabilities:
    ndps_pillar_dates.append(dates['date'])
    
credit_curve = CreditCurve(ndps_pillar_dates,
                           pillar_ndps)

credit_curve.ndp(date(2025,10,31))

0.833046076369603

Now that we have all the ingredients we can compute the market values of the portfolio of bonds.

In [19]:
from datetime import date 
from dateutil.relativedelta import relativedelta 

class BondPricer: 
    
    def __init__(self, pricing_date, nominal, start_date, end_date,
                 current_coupon, margin, recovery, tenor_months=6):
        
        self.pricing_date = pricing_date 
        self.nominal = nominal
        self.start_date = start_date 
        self.end_date = end_date 
        self.current_coupon = current_coupon
        self.margin = margin 
        self.recovery = recovery 
        self.tenor_months = tenor_months 
        

    def payment_dates(self):
        dates = []
        future_dates = []

        n_months = ((self.end_date.year - self.start_date.year) * 12) + \
                    (self.end_date.month - self.start_date.month)

        for n in range(0, n_months, self.tenor_months):
            dates.append(self.start_date + relativedelta(months=n))
        dates.append(self.end_date)

        for i in range(len(dates)):

            if dates[i] > self.pricing_date: 
                future_dates.append(dates[i])

        return future_dates
        
    def mark_to_market(self, discounting_curve, forward_curve, credit_curve):
        
        bond_dates = self.payment_dates()
        npv_coupons_if_no_default = 0
        recovery_if_early_default = 0 
    
        for i in range(1, len(bond_dates)):
            if bond_dates[i] > today:
                npv_coupons_if_no_default += self.nominal * \
                    (((bond_dates[i] - bond_dates[i-1]).days / 360) *
                        discounting_curve.df(bond_dates[i]) *
                        (forward_curve.forward_rate(bond_dates[i-1]) + self.margin) * 
                        credit_curve.ndp(bond_dates[i]))
                    
        notional_reimbursement_if_no_default = self.nominal * \
                    (discounting_curve.df(bond_dates[-1])*
                     credit_curve.ndp(bond_dates[-1]))
        
        
        first_coupon = (((bond_dates[0] - today).days / 360)*
                          discounting_curve.df(bond_dates[0]) *
                          self.current_coupon * credit_curve.ndp(bond_dates[0]))

        npv_coupons_if_no_default += first_coupon
        
        d = self.pricing_date
        while d <= bond_dates[-1]:
            recovery_if_early_default += ((discounting_curve.df(d)) *
                                         (credit_curve.ndp(d) - 
                                          credit_curve.ndp(d+relativedelta(days=1))))
            d += relativedelta(days=1)  

        recovery_if_early_default = recovery_if_early_default * self.recovery * self.nominal
        MTM =  npv_coupons_if_no_default + notional_reimbursement_if_no_default + recovery_if_early_default
        
        return MTM

In [24]:
ptf_price= 0.0

for quote in bonds_to_price:   
    ptf = BondPricer(today, quote['nominal'],
                     quote['start_date'], quote['end_date'],
                     quote['current_coupon'], quote['margin'], quote['recovery'])
    ptf_price += ptf.mark_to_market(eonia_curve, 
                                    forward_rate_curve, 
                                    credit_curve)

print ("The market value of the portfolio is {:.2f}".format(ptf_price))

The market value of the portfolio is 78927121.13
